# 03 - Preparación de Validación con Psiquiatras (Post-Denoising)

**Objetivo:** Generar un set de casos para validar decisiones críticas del pipeline, específicamente el proceso de **Denoising** y la calidad de las etiquetas.

**Cambio de Estrategia:**
Anteriormente nos enfocábamos en detectar "ruido administrativo". Ahora que hemos implementado un **Denoising Rule-Based** (Notebook 03), el objetivo es validar si este proceso es correcto:
1. **Validar Exclusiones (Falsos Negativos del Denoising):** ¿Estamos eliminando casos que SÍ tienen información clínica relevante?
2. **Validar Inclusiones (Falsos Positivos del Denoising):** ¿Estamos incluyendo casos que siguen siendo ruido?
3. **Validar Negaciones:** Decidimos incluir síntomas negados (ej: "niega ansiedad"). ¿Están de acuerdo los psiquiatras?
4. **Validar Vocabulario:** Identificar términos paraguayos en los casos que el sistema no detectó.

In [1]:
import sys
import pandas as pd
import numpy as np
from pathlib import Path

# Importar utilidades compartidas
current_dir = Path.cwd()
if current_dir.name == "notebooks":
    sys.path.append(str(current_dir.parent))
else:
    sys.path.append(str(current_dir))

try:
    from notebooks.utils_shared import setup_paths
except ImportError:
    sys.path.append(str(current_dir))
    from utils_shared import setup_paths

paths = setup_paths()
DATA_PATH = paths['DATA_PATH']
SPLITS_PATH = paths['SPLITS_PATH']

print(f"[OK] Paths configurados.")

[OK] Paths configurados.


## 1. Cargar Datos: Original vs Denoised

In [2]:
# Cargar dataset base (todos los casos)
df_base = pd.read_csv(SPLITS_PATH / 'dataset_base.csv')

# Cargar dataset denoised (solo los que pasaron el filtro)
try:
    df_denoised = pd.read_csv(SPLITS_PATH / 'train_denoised.csv')
    denoised_ids = set(df_denoised['row_id'])
except FileNotFoundError:
    print("[ERROR] No se encontró train_denoised.csv. Ejecuta 03_rule_based_denoising.ipynb primero.")
    raise

# Identificar excluidos
df_base['is_kept'] = df_base['row_id'].isin(denoised_ids)
df_excluded = df_base[~df_base['is_kept']].copy()
df_included = df_base[df_base['is_kept']].copy()

print(f"Total casos: {len(df_base)}")
print(f"Casos MANTENIDOS (Clinical Signal): {len(df_included)} ({len(df_included)/len(df_base):.1%})")
print(f"Casos EXCLUIDOS (Noise/No Signal): {len(df_excluded)} ({len(df_excluded)/len(df_base):.1%})")

Total casos: 3131
Casos MANTENIDOS (Clinical Signal): 993 (31.7%)
Casos EXCLUIDOS (Noise/No Signal): 2138 (68.3%)


## 2. Selección de Casos para Validación

Seleccionaremos 25 casos distribuidos estratégicamente:

In [3]:
casos_seleccionados = []

# --- GRUPO 1: Excluidos "Sospechosos" (Posibles Falsos Negativos) ---
# Buscamos casos excluidos que sean largos (>50 chars). Si son largos y se borraron,
# quizás contienen vocabulario paraguayo no detectado.
excluidos_largos = df_excluded[df_excluded['texto'].str.len() > 50]
if not excluidos_largos.empty:
    sample_excluidos = excluidos_largos.sample(min(10, len(excluidos_largos)), random_state=42)
    sample_excluidos['Grupo'] = 'Excluido_Sospechoso'
    sample_excluidos['Motivo'] = 'Texto largo pero borrado por denoising. ¿Contiene síntomas no detectados?'
    casos_seleccionados.append(sample_excluidos)

# --- GRUPO 2: Inclusiones con Negación (Validar Criterio) ---
# Buscamos casos mantenidos que contengan "no " o "niega".
# Queremos confirmar si los psiquiatras consideran esto evidencia clínica.
mask_negacion = df_included['texto'].str.lower().str.contains(r'\b(no|niega|sin)\b', regex=True, na=False)
incluidos_negados = df_included[mask_negacion]
if not incluidos_negados.empty:
    sample_negados = incluidos_negados.sample(min(8, len(incluidos_negados)), random_state=42)
    sample_negados['Grupo'] = 'Inclusion_Negacion'
    sample_negados['Motivo'] = 'Contiene negaciones. ¿Es síntoma (falta insight) o ausencia real?'
    casos_seleccionados.append(sample_negados)

# --- GRUPO 3: Casos Cortos Mantenidos (Posibles Falsos Positivos) ---
# Casos cortos que pasaron el filtro. ¿Son realmente útiles?
incluidos_cortos = df_included[df_included['texto'].str.len() < 100]
if not incluidos_cortos.empty:
    sample_cortos = incluidos_cortos.sample(min(7, len(incluidos_cortos)), random_state=42)
    sample_cortos['Grupo'] = 'Inclusion_Corta'
    sample_cortos['Motivo'] = 'Texto corto mantenido. ¿Es suficiente evidencia?'
    casos_seleccionados.append(sample_cortos)

# Unificar
df_validacion = pd.concat(casos_seleccionados).head(25)

# Preparar columnas para el Excel
cols_export = ['row_id', 'Grupo', 'etiqueta', 'texto', 'Motivo']
df_export = df_validacion[cols_export].copy()
df_export['Validacion_Psiquiatra'] = ''  # Espacio para que escriban
df_export['Comentarios'] = ''

print(f"Seleccionados {len(df_export)} casos para validación.")
print(df_export['Grupo'].value_counts())

Seleccionados 22 casos para validación.
Grupo
Excluido_Sospechoso    10
Inclusion_Negacion      8
Inclusion_Corta         4
Name: count, dtype: int64


/var/folders/25/fy01l91x3gj63g090ghxj7000000gn/T/ipykernel_44429/3827656830.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_negacion = df_included['texto'].str.lower().str.contains(r'\b(no|niega|sin)\b', regex=True, na=False)


## 3. Exportar a Excel

In [4]:
output_file = DATA_PATH / "VALIDACION_PSIQUIATRAS_POST_DENOISING.xlsx"
try:
    df_export.to_excel(output_file, index=False)
    print(f"[OK] Excel generado: {output_file}")
except ImportError:
    print("[ERROR] Falta openpyxl. Instalando...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "openpyxl"])
    df_export.to_excel(output_file, index=False)
    print(f"[OK] Excel generado: {output_file}")

[OK] Excel generado: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/VALIDACION_PSIQUIATRAS_POST_DENOISING.xlsx
